In [ ]:
import pandas as pd
import numpy as np
from nptdms import TdmsFile
import os
import matplotlib.pyplot as plt

# PATHS
BASE = r"c:\Users\es-sabar\Documents\PreTest\Moto_04112025_chicane_sec"
TDMS_PATH = os.path.join(BASE, "labview_data", "Moto_Chicane_50.tdms")
XSENS_P1 = os.path.join(BASE, "Moto_chicane_TXT", "Moto_Chicane_50_P1.txt")
XSENS_P2 = os.path.join(BASE, "Moto_chicane_TXT", "Moto_Chicane_50_P2.txt")

def load_xsens_head(path, name):
    print(f"\n--- XSENS {name} ---")
    h_idx = None
    try:
        with open(path, 'r', errors='ignore') as f:
            for i, l in enumerate(f):
                if l.strip().startswith('PacketCounter'): h_idx=i; break
        if h_idx is None:
             with open(path, 'r', errors='ignore') as f:
                for i, l in enumerate(f): 
                    if 'UTC_Year' in l: h_idx=i; break
        
        # Robust Read
        try: df = pd.read_csv(path, sep='\t', header=h_idx, nrows=50000)
        except: df = pd.read_csv(path, sep=r'\s+', header=h_idx, nrows=50000)
        
        df.columns = df.columns.str.strip()
        
        req = ['UTC_Year', 'UTC_Month', 'UTC_Day', 'UTC_Hour', 'UTC_Minute', 'UTC_Second', 'UTC_Nano']
        if not set(req).issubset(df.columns):
            print("COLUMNS MISSING:", df.columns)
            return None
            
        ts = pd.to_datetime(df[req[:-1]].astype(int).rename(columns={'UTC_Year':'year','UTC_Month':'month','UTC_Day':'day','UTC_Hour':'hour','UTC_Minute':'minute','UTC_Second':'second'}))
        ts += pd.to_timedelta(df['UTC_Nano'], unit='ns')
        
        print(f"Start: {ts.min()}")
        print(f"End  : {ts.max()}")
        return ts.min()
    except Exception as e:
        print(f"Error loading Xsens: {e}")
        return None
    
def inspect_tdms(path, group_name):
    print(f"\n--- TDMS Group {group_name} ---")
    tdms = TdmsFile.read(path)
    found=False
    for g in tdms.groups():
        if g.name == group_name:
            found=True
            
            # METADATA CHECK (Start Time)
            meta_start = None
            if 'Edges_RoueAR' in g:
                c = g['Edges_RoueAR']
                print("CHAN PROPS:", c.properties)
                if 'wf_start_time' in c.properties:
                     meta_start = pd.to_datetime(c.properties['wf_start_time']).tz_localize(None)
            
            if not meta_start and 'wf_start_time' in tdms.properties:
                 meta_start = pd.to_datetime(tdms.properties['wf_start_time']).tz_localize(None)
            
            print(f"METADATA START TIME: {meta_start}")

            if 'Edges_RoueAR' in g:
                chan = g['Edges_RoueAR']
                data = chan[:]
                print(f"Data Len: {len(data)}")
                
                plt.figure(figsize=(10,3))
                plt.plot(data)
                plt.title(f"Edges_RoueAR - {group_name}")
                plt.show()
                
                # RESET CHECK
                diffs = np.diff(data)
                resets = np.where(diffs < -100)[0]
                if len(resets) > 0:
                    print(f"!!! RESET DETECTED at {resets[0]} !!!")
                else:
                    print("No Reset Detected.")
    if not found:
        print("Group not found.")

print("Analyzing...")
ts_xs_1 = load_xsens_head(XSENS_P1, "P1")
inspect_tdms(TDMS_PATH, "P1")

ts_xs_2 = load_xsens_head(XSENS_P2, "P2")
inspect_tdms(TDMS_PATH, "P2")